In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Send a GET request to the IMDb URL
url = "https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1"
response = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")

# Extract data for the first CSV file
data_file1 = []
movies = soup.find_all("div", class_="lister-item mode-advanced")
for movie in movies:
    movie_name_element = movie.find("h3", class_="lister-item-header").find("a")
    movie_name = movie_name_element.text.strip()
    
    director_element = movie.find_all("p")[2].find_all("a")
    directors = [director.text.strip() for director in director_element]
    
    duration_element = movie.find("span", class_="runtime")
    duration = duration_element.text.strip() if duration_element else ""
    
    year_element = movie.find("span", class_="lister-item-year")
    year = year_element.text.strip()[1:-1] if year_element else ""
    
    ratings_element = movie.find("div", class_="ratings-imdb-rating")
    ratings = ratings_element.text.strip() if ratings_element else ""
    
    metascore_element = movie.find("span", class_="metascore")
    metascore = metascore_element.text.strip() if metascore_element else ""
    
    data_file1.append([movie_name, ", ".join(directors), duration, year, ratings, metascore])

# Save data to the first CSV file
df1 = pd.DataFrame(data_file1, columns=["Movie Name", "Directors", "Duration", "Year", "Ratings", "Metascore"])
df1["Sno"] = df1.index + 1
df1.to_csv("movies_data_1.csv", index=False)

# Extract data for the second CSV file
data_file2 = []
for movie in movies:
    movie_name_element = movie.find("h3", class_="lister-item-header").find("a")
    movie_name = movie_name_element.text.strip()
    
    stars_element = movie.find("p", class_="text-muted").find("span", class_="certificate")
    stars = stars_element.text.strip() if stars_element else ""
    
    votes_element = movie.find("p", class_="sort-num_votes-visible").find("span", attrs={"name": "nv"})
    votes = votes_element["data-value"] if votes_element else ""
    
    genre_element = movie.find("p", class_="text-muted").find("span", class_="genre")
    genre = genre_element.text.strip() if genre_element else ""
    
    gross_element = movie.find("p", class_="sort-num_votes-visible").find_all("span", attrs={"name": "nv"})[-1]
    gross = gross_element["data-value"] if gross_element else ""
    
    popularity_element = movie.find("span", class_="popularityTrend")
    popularity = popularity_element.text.strip() if popularity_element else ""
    
    certification_element = movie.find("p", class_="text-muted").find("span", class_="certificate")
    certification = certification_element.text.strip() if certification_element else ""
    
    data_file2.append([movie_name, stars, votes, genre, gross, popularity, certification])

# Save data to the second CSV file
df2 = pd.DataFrame(data_file2, columns=["Movie Name", "Stars", "Votes", "Genre", "Gross Collection", "Popularity", "Certification"])
stars_columns = df2["Stars"].str.split(",", expand=True).columns
df2[stars_columns] = df2["Stars"].str.split(",", expand=True)
genre_columns = df2["Genre"].str.split(",", expand=True).columns
df2[genre_columns] = df2["Genre"].str.split(",", expand=True)
df2.drop(["Stars", "Genre"], axis=1, inplace=True)
df2.to_csv("movies_data_2.csv", index=False)


In [22]:
import sqlite3
import pandas as pd

# Create a SQLite database connection
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Create Table: movies_data
create_table_query = '''
CREATE TABLE IF NOT EXISTS movies_data (
    Sno INTEGER PRIMARY KEY,
    MovieName TEXT,
    Director TEXT,
    Duration INTEGER,
    Genre TEXT,
    Ratings REAL
)
'''
cursor.execute(create_table_query)

# Insert data into movies_data table
data = [
    (1, 'Inception', 'Christopher Nolan', 148, 'Action, Adventure, Sci-Fi', 8.8),
    (2, 'The Dark Knight Rises', 'Christopher Nolan', 164, 'Action, Adventure', 8.4),
    (3, 'Dawn of the Planet of the Apes', 'Matt Reeves', 130, 'Action, Adventure, Drama, Sci-Fi', 7.6),
    (4, 'War for the Planet of the Apes', 'Matt Reeves', 140, 'Action, Adventure, Drama, Sci-Fi', 7.4),
    (5, 'Cloverfield', 'Matt Reeves', 85, 'Action, Horror, Sci-Fi, Thriller', 7.0)
]

insert_query = '''
INSERT INTO movies_data (Sno, MovieName, Director, Duration, Genre, Ratings)
VALUES (?, ?, ?, ?, ?, ?)
'''

cursor.executemany(insert_query, data)
conn.commit()

# Execute a sample query
query = "SELECT * FROM movies_data"
df = pd.read_sql_query(query, conn)
print(df)

# Close the database connection
conn.close()


   Sno                       MovieName           Director  Duration  \
0    1                       Inception  Christopher Nolan       148   
1    2           The Dark Knight Rises  Christopher Nolan       164   
2    3  Dawn of the Planet of the Apes        Matt Reeves       130   
3    4  War for the Planet of the Apes        Matt Reeves       140   
4    5                     Cloverfield        Matt Reeves        85   

                              Genre  Ratings  
0         Action, Adventure, Sci-Fi      8.8  
1                 Action, Adventure      8.4  
2  Action, Adventure, Drama, Sci-Fi      7.6  
3  Action, Adventure, Drama, Sci-Fi      7.4  
4  Action, Horror, Sci-Fi, Thriller      7.0  
